In [1]:
#instalo/actualizo
#! pip3 install --upgrade google-cloud-aiplatform


In [50]:
import re
import pandas as pd
from google.cloud import bigquery
#import vertexai 
#from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part, Content
#import vertexai.preview.generative_models as generative_models 
import sys
import os
from fuzzywuzzy import process, fuzz

In [51]:
# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project='ldp-data-wyscout')


In [52]:
###ME TRAIGO LOS DATOS DE LA TABLA DE CAPOLOGY

# Consulta SQL
query2 = """SELECT * FROM `ldp-data-wyscout.capology.salarios_copy`"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df1 = client.query(query2).to_dataframe()
    print(df1.head())
except Exception as e:
    print(f"An error occurred: {e}")


                  region           pais liga                    club  \
0  Central/North America  United States  MLS          Atlanta United   
1  Central/North America  United States  MLS  New England Revolution   
2  Central/North America  United States  MLS      Philadelphia Union   
3  Central/North America  United States  MLS        Seattle Sounders   
4  Central/North America  United States  MLS            Nashville SC   

   weeklyavg  annualavg   moneda  domestic_competition_divisionLevel  
0       6451     335452  dolares                                   1  
1       9209     478875  dolares                                   1  
2       6990     363486  dolares                                   1  
3      10668     554728  dolares                                   1  
4      11651     605862  dolares                                   1  


In [8]:
df1.head(20)

,region,pais,liga,club,weeklyavg,annualavg,moneda,domestic_competition_divisionLevel
0,Central/North America,United States,MLS,Atlanta United,6451,335452,dolares,1
1,Central/North America,United States,MLS,New England Revolution,9209,478875,dolares,1
2,Central/North America,United States,MLS,Philadelphia Union,6990,363486,dolares,1
3,Central/North America,United States,MLS,Seattle Sounders,10668,554728,dolares,1
4,Central/North America,United States,MLS,Nashville SC,11651,605862,dolares,1
5,Central/North America,United States,MLS,Minnesota United,6842,355810,dolares,1
6,Central/North America,United States,MLS,Orlando City,8178,425248,dolares,1
7,Central/North America,Canada,MLS,Toronto FC,13244,688666,dolares,1
8,Central/North America,United States,MLS,FC Cincinnati,9171,476900,dolares,1
9,Central/North America,United States,MLS,Colorado Rapids,8771,456102,dolares,1


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   region                              570 non-null    object
 1   pais                                570 non-null    object
 2   liga                                570 non-null    object
 3   club                                570 non-null    object
 4   weeklyavg                           566 non-null    Int64 
 5   annualavg                           566 non-null    Int64 
 6   moneda                              570 non-null    object
 7   domestic_competition_divisionLevel  570 non-null    Int64 
dtypes: Int64(3), object(5)
memory usage: 37.4+ KB


In [10]:
#df1.dropna(inplace=True)

In [46]:
dftext = df1.to_csv('capology.csv', index=False)

In [47]:
###FILTRO LA DE DICT TEAMS POR team_type="parent" and category="default" and gender = 'male' Y ADEMAS POR LOS PAISES QUE ESTAN EN LA DE CAPOLOGY PARA ACHICAR REGISTROS

query = """SELECT DISTINCT
    t.wyId, 
    t.name, 
    t.officialName, 
    t.area_name, 
    t.domestic_competition_name,
    t.region,
    t.domestic_competition_divisionLevel
FROM 
    ldp-data-wyscout.wyscout_curated.dict_teams t
    inner join ldp-data-wyscout.wyscout_curated.dict_competition_list c on c.wyId = t.domestic_competition_id
and c.band <= 6
WHERE 
    t.team_type = "parent"
    AND t.category = "default"
    AND t.gender = 'male'
    and t.domestic_competition_divisionLevel <= 4
    AND t.area_name IN (
        SELECT DISTINCT c.pais
        FROM ldp-data-wyscout.capology.salarios_copy c
    );
 """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df2 = client.query(query).to_dataframe()
    print(df2.head())
except Exception as e:
    print(f"An error occurred: {e}")

    wyId             name          officialName    area_name  \
0     59              ARC  Alphense Racing Club  Netherlands   
1  13992         Orenburg           FK Orenburg       Russia   
2   3252      Juve Stabia        SS Juve Stabia        Italy   
3  26152  Felgueiras 1932    FC Felgueiras 1932     Portugal   
4  15415    Santos Laguna    Club Santos Laguna       Mexico   

  domestic_competition_name                 region  \
0            Vierde Divisie              Europe EU   
1            Premier League            Europe rest   
2                   Serie B              Europe EU   
3              Segunda Liga              Europe EU   
4                   Liga MX  Central/North America   

   domestic_competition_divisionLevel  
0                                   4  
1                                   1  
2                                   2  
3                                   2  
4                                   1  


In [14]:
df2.head()

,wyId,name,officialName,area_name,domestic_competition_name,region,domestic_competition_divisionLevel
0,13508,Wisła Płock,Wisła Płock SA,Poland,I Liga,Europe EU,2
1,8746,Austria Lustenau,SC Austria Lustenau,Austria,2. Liga,Europe EU,2
2,13512,Górnik Zabrze,KS Górnik Zabrze,Poland,Ekstraklasa,Europe EU,1
3,7510,Hobro,Hobro IK,Denmark,1st Division,Europe EU,2
4,70804,Inter de Querétaro FC,Inter de Querétaro Futbol Club,Mexico,Liga Premier Serie A,Central/North America,3


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013 entries, 0 to 2012
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   wyId                                2013 non-null   Int64 
 1   name                                2013 non-null   object
 2   officialName                        2013 non-null   object
 3   area_name                           2013 non-null   object
 4   domestic_competition_name           2013 non-null   object
 5   region                              2013 non-null   object
 6   domestic_competition_divisionLevel  2013 non-null   Int64 
dtypes: Int64(2), object(5)
memory usage: 114.1+ KB


In [48]:
dftext2 = df2.to_csv('dict_teams.csv',index=False)

DOCU FUZZY  https://pypi.org/project/fuzzywuzzy/

In [53]:
fuzz.ratio("this is a test", "this is a test!")

97

In [54]:
fuzz.partial_ratio("this is a test", "this is a test!")
 

100

In [55]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

In [57]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

In [58]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100

UNION DE DF CON LOS PAISES EN INGLES DE LA TABLA SALARIOS_COPY CON DICT TEAMS USANDO FUZZY----


In [37]:
# Normalizar los nombres en ambas tablas
df1['club'] = df1['club'].str.lower().str.strip()
df1['liga'] = df1['liga'].str.lower().str.strip()
df1['pais'] = df1['pais'].str.lower().str.strip()
df1['region'] = df1['region'].str.lower().str.strip()

df2['name'] = df2['name'].str.lower().str.strip()
df2['officialName'] = df2['officialName'].str.lower().str.strip()
df2['domestic_competition_name'] = df2['domestic_competition_name'].str.lower().str.strip()
df2['area_name'] = df2['area_name'].str.lower().str.strip()
df2['region'] = df2['region'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia en una lista
def get_best_match(value, choices):
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 50 else None

# Encontrar el ID de los clubes en la segunda tabla
df1['wyId'] = df1['club'].apply(lambda x: get_best_match(x, df2['name'].tolist()))

# Mapear los nombres encontrados a los IDs reales en df2
name_to_wyId = dict(zip(df2['name'], df2['wyId']))
df1['wyId'] = df1['wyId'].map(name_to_wyId)

# Merge basado en la columna 'wyId' para obtener la información completa
merged_df = pd.merge(df1, df2, left_on='wyId', right_on='wyId', how='inner', suffixes=('_tabla1', '_tabla2'))

# Verifica cómo se ve el merge
print(merged_df[['club', 'wyId', 'name', 'officialName']].head(10))

# Filtrar columnas y organizar el DataFrame final
final_columns2 = [
    'region_tabla1', 'pais', 'liga', 'club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'domestic_competition_name', 'region_tabla2'
]

# Asegúrate de que las columnas existen en el DataFrame
missing_columns = [col for col in final_columns2 if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")

# Seleccionar solo las columnas presentes en merged_df
final_columns2_existing = [col for col in final_columns2 if col in merged_df.columns]
final_df2 = merged_df[final_columns2_existing]

# Renombrar columnas para claridad
final_df2.rename(columns={
    'region_tabla1': 'region1',
    'pais': 'pais',
    'liga': 'liga',
    'club': 'club',
    'weeklyavg': 'weeklyavg',
    'annualavg': 'annualavg',
    'moneda': 'moneda',
    'wyId': 'wyId',
    'name': 'name',
    'officialName': 'officialName',
    'area_name': 'area_name',
    'domestic_competition_name': 'domestic_competition_name',
    'region_tabla2': 'region2'
}, inplace=True)

# Guardar o imprimir el DataFrame final
final_df2.to_csv('tabla_combinada.csv', index=False)
print(final_df2)


                     club   wyId                name            officialName
0          atlanta united  58165      atlanta united       atlanta united fc
1  new england revolution   7854         new england  new england revolution
2      philadelphia union   8061  philadelphia union      philadelphia union
3        seattle sounders   8067    seattle sounders     seattle sounders fc
4            nashville sc  30149        nashville sc            nashville sc
5        minnesota united   8095    minnesota united     minnesota united fc
6            orlando city   8068        orlando city         orlando city sc
7              toronto fc  14995             toronto              toronto fc
8           fc cincinnati  37892          cincinnati           fc cincinnati
9         colorado rapids   7853     colorado rapids         colorado rapids
                   region1           pais         liga  \
0    central/north america  united states          mls   
1    central/north america  united st

C:\Users\flori\AppData\Local\Temp\ipykernel_26872\184841794.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2.rename(columns={


In [38]:
final_df2['wyId'] = final_df2['wyId'].astype(int)


C:\Users\flori\AppData\Local\Temp\ipykernel_26872\3153981150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['wyId'] = final_df2['wyId'].astype(int)


In [76]:
# Renombrar columnas duplicadas sino me da error al intentar subirlo a BQ
#final_df2.columns = [
#    'region_tabla1','pais','liga','club', 
#    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
#    'officialName', 'area_name','domestic_competition_name' ,'region_tabla2'
#]


In [39]:
from google.cloud import bigquery

# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.fuzzy2'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(final_df2, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.fuzzy2.


ahora priorizando pais

In [33]:
from fuzzywuzzy import process, fuzz
import pandas as pd

# Normalizar las columnas
df1['club'] = df1['club'].str.lower().str.strip()
df1['pais'] = df1['pais'].str.lower().str.strip()
df1['region'] = df1['region'].str.lower().str.strip()

df2['name'] = df2['name'].str.lower().str.strip()
df2['officialName'] = df2['officialName'].str.lower().str.strip()
df2['area_name'] = df2['area_name'].str.lower().str.strip()
df2['region'] = df2['region'].str.lower().str.strip()

# Función para encontrar la mejor coincidencia
def get_best_match(value, choices):
    match, score = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= 60 else None

# Inicializar una lista para almacenar los IDs
df1['wyId'] = None

# Priorizar los matchings
for column in ['club', 'pais', 'region']:
    if column == 'club':
        df1['wyId'] = df1['club'].apply(lambda x: get_best_match(x, df2['name'].tolist()))
    elif column == 'pais':
        # Solo buscar si no hay match en club
        df1.loc[df1['wyId'].isnull(), 'wyId'] = df1.loc[df1['wyId'].isnull(), 'pais'].apply(
            lambda x: get_best_match(x, df2['area_name'].tolist())
        )
    elif column == 'region':
        # Solo buscar si no hay match en pais
        df1.loc[df1['wyId'].isnull(), 'wyId'] = df1.loc[df1['wyId'].isnull(), 'region'].apply(
            lambda x: get_best_match(x, df2['region'].tolist())
        )

# Mapear los nombres encontrados a los IDs reales en df2
name_to_wyId = dict(zip(df2['name'], df2['wyId']))
df1['wyId'] = df1['wyId'].map(name_to_wyId)

# Merge basado en la columna 'wyId'
merged_df = pd.merge(df1, df2, on='wyId', how='inner', suffixes=('_tabla1', '_tabla2'))

# Verifica cómo se ve el merge
print(merged_df[['club', 'wyId', 'name', 'officialName']].head(10))

# Filtrar columnas y organizar el DataFrame final
final_columns2 = [
    'region_tabla1', 'pais', 'club', 
    'weeklyavg', 'annualavg', 'moneda', 'wyId', 'name', 
    'officialName', 'area_name', 'region_tabla2'
]

# Asegúrate de que las columnas existen en el DataFrame
missing_columns = [col for col in final_columns2 if col not in merged_df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")

# Seleccionar solo las columnas presentes en merged_df
final_columns2_existing = [col for col in final_columns2 if col in merged_df.columns]
final_df22 = merged_df[final_columns2_existing]

# Renombrar columnas para claridad
final_df22.rename(columns={
    'region_tabla1': 'region1',
    'pais': 'pais',
    'club': 'club',
    'weeklyavg': 'weeklyavg',
    'annualavg': 'annualavg',
    'moneda': 'moneda',
    'wyId': 'wyId',
    'name': 'name',
    'officialName': 'officialName',
    'area_name': 'area_name',
    'region_tabla2': 'region2'
}, inplace=True)

# Guardar o imprimir el DataFrame final
final_df22.to_csv('tabla_combinada2.csv', index=False)
print(final_df22)


                     club     wyId                name            officialName
0          atlanta united  58165.0      atlanta united       atlanta united fc
1  new england revolution   7854.0         new england  new england revolution
2      philadelphia union   8061.0  philadelphia union      philadelphia union
3        seattle sounders   8067.0    seattle sounders     seattle sounders fc
4            nashville sc  30149.0        nashville sc            nashville sc
5        minnesota united   8095.0    minnesota united     minnesota united fc
6            orlando city   8068.0        orlando city         orlando city sc
7              toronto fc  14995.0             toronto              toronto fc
8           fc cincinnati  37892.0          cincinnati           fc cincinnati
9         colorado rapids   7853.0     colorado rapids         colorado rapids
                   region1           pais                    club  weeklyavg  \
0    central/north america  united states          

C:\Users\flori\AppData\Local\Temp\ipykernel_26872\4125227688.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df22.rename(columns={


In [34]:
from google.cloud import bigquery

# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.fuzzy'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Reemplaza la tabla si ya existe
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(final_df2, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.fuzzy.
